# Quantitative evaluation

Use ArFace metric and Emotion metric to compare the performance of : 
    - e4e initialization vs hybrid initialization
    - og loss vs custom loss

In [8]:
from argparse import Namespace
import time
import sys
import numpy as np
from PIL import Image
import torch
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from torch.amp import autocast
from tqdm import tqdm
import clip
import torch.nn.functional as F

from utils.common import tensor2im
from utils.alignment import run_alignment
from models.psp import pSp  # we use the pSp framework to load the e4e encoder.
from criteria.clip_loss import CLIPLoss
from criteria.id_loss import IDLoss

%load_ext autoreload
%autoreload 2
%matplotlib inline

device = 'cuda' if torch.cuda.is_available() else 'cpu'
RESIZE_DIMS = (256, 256)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
# Define the arguments for the e4e encoder

pretrained_pSp_path = 'pretrained_models/e4e_ffhq_encode.pt'

e4e_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])]
)

In [3]:
# Load the pSp model

ckpt = torch.load(pretrained_pSp_path, map_location='cpu')
opts = ckpt['opts']
opts['checkpoint_path'] = pretrained_pSp_path
opts['ir_se50_weights'] = 'pretrained_models/model_ir_se50.pth'
opts= Namespace(**opts)
latent_avg = ckpt["latent_avg"]

psp_model = pSp(opts)
psp_model.eval()
psp_model.cuda()

print('Model successfully loaded!')

/tmp/ipykernel_333685/3793385928.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(pretrained_pSp_path, map_location='cpu')


Loading e4e over the pSp framework from checkpoint: pretrained_models/e4e_ffhq_encode.pt


/users/eleves-b/2022/rodrigue.reibel/emotion_generation/models/psp.py:44: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(self.opts.checkpoint_path, map_loca

Model successfully loaded!


In [ ]:
# Load the emotion model

from emotionmmodel.networks.DDAM import DDAMNet
emotion_model = DDAMNet(num_class=7, num_head=2, pretrained=False)
path = "pretrained_models/affecnet7_epoch19_acc0.671.pth"
checkpoint = torch.load(path, map_location=device)
emotion_model.load_state_dict(checkpoint['model_state_dict'])
emotion_model.to(device)
emotion_model.eval()    
val_transform = transforms.Compose([
        transforms.Resize((112, 112)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])]) 

/tmp/ipykernel_333685/3776924524.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(path, map_location=device)


In [24]:
import os

input_images_path = "faces_dataset_very_small"
N_images = len(os.listdir(input_images_path))
images = np.array(os.listdir(input_images_path))[np.random.randint(0, len(os.listdir(input_images_path)), N_images)]


In [26]:
# Parameters for inversion
optimization_steps_inv = 50
lambda_percept_inv = 1
lambda_L2_inv = 0.5
lr_inv = 0.01 
final_lr_inv = 0.001

# Parameters for editing
optimization_steps_edit = 50
lambda_L2_edit = 0.01
lambda_ID_edit = 0.01
lambda_E_edit = 0.01
lr_edit = 0.01
final_lr_edit = 0.001
truncation_edit = 0.9

id_loss = IDLoss(opts)
L2_loss = torch.nn.MSELoss().to(device)
clip_loss = CLIPLoss(opts)
emotion_loss = torch.nn.CrossEntropyLoss()

G = psp_model.decoder
prompt = "A person smiling"
# class7_names = ['Neutral', 'Happy', 'Sad', 'Surprise', 'Fear', 'Disgust', 'Angry']
goal_distribution = torch.tensor([[0., 1., 0., 0., 0., 0., 0.]], device=device)

Loading ResNet ArcFace


In [27]:
sims_e4e = []
sims_hybrid = []
sims_e4e_edit = []
sims_hybrid_edit = []
emo_scores_e4e = []
emo_scores_hybrid = []

for i_image, image_path in enumerate(images):
    print(f"*** Processing image {i_image + 1}/{N_images} ***")
    image = run_alignment(os.path.join(input_images_path, image_path)).resize(RESIZE_DIMS)
    input_image = e4e_transform(image)
    reference_image = transforms.Resize((1024, 1024))(input_image).unsqueeze(0).cuda()


    with torch.no_grad():
        e4e_inverted_latent = psp_model.encoder(input_image.unsqueeze(0).to(device))
    if psp_model.opts.start_from_latent_avg:
                    if e4e_inverted_latent.ndim == 2:
                        e4e_inverted_latent = e4e_inverted_latent + psp_model.latent_avg.repeat(e4e_inverted_latent.shape[0], 1, 1)[:, 0, :]
                    else:
                        e4e_inverted_latent = e4e_inverted_latent + psp_model.latent_avg.repeat(e4e_inverted_latent.shape[0], 1, 1)


    w_n = e4e_inverted_latent.clone().detach().requires_grad_(True).to(device) # initialization with e4e output

    optimizer = torch.optim.Adam([w_n], lr=lr_inv)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=optimization_steps_inv, eta_min=final_lr_inv)

 

    pbar = tqdm(range(optimization_steps_inv))

    imgs = []
    losses = []


    for i in pbar:

        with autocast("cuda"):
            
            optimizer.zero_grad()

            img_gen, _ = G([w_n], input_is_latent=True, randomize_noise=False)

            i_loss = id_loss(img_gen, reference_image)[0]
            l2_loss = L2_loss(img_gen, reference_image)

            loss = i_loss * lambda_percept_inv + l2_loss * lambda_L2_inv 
            losses.append(loss.item())
            pbar.set_description(f"Loss: {loss.item():.4f}")

            loss.backward()
            optimizer.step()
            scheduler.step()

    inverted_latent = w_n.detach()

    # Compute ID metric between inverted images and original image

    with torch.no_grad():
        e4e_inverted_image, _ = G([e4e_inverted_latent], input_is_latent=True, randomize_noise=False)
        inverted_image = img_gen.detach()

    sim_e4e = id_loss(e4e_inverted_image, reference_image)[0]
    sim_hybrid = id_loss(inverted_image, reference_image)[0]

    sims_e4e.append(sim_e4e.item())
    sims_hybrid.append(sim_hybrid.item())

    ##### Editing evaluation #####

    text_inputs = torch.cat([clip.tokenize(prompt)]).cuda()

    for latent, sims, emo_scores in zip([e4e_inverted_latent, inverted_latent], [sims_e4e_edit, sims_hybrid_edit], [emo_scores_e4e, emo_scores_hybrid]):

    # Initialization
        w_start = latent

        w_n = w_start.clone().detach().requires_grad_(True).to(device)
        optimizer = torch.optim.Adam([w_n], lr=lr_edit)
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=optimization_steps_edit, eta_min=final_lr_edit)


        with torch.no_grad():
            img_orig, _ = G([w_n], input_is_latent=True)

        pbar = tqdm(range(optimization_steps_edit))

        imgs = []
        losses = []

        # Optimization loop

        for i in pbar:

            with autocast("cuda"):
                
                optimizer.zero_grad()

                img_gen, _ = G([w_n], input_is_latent=True, randomize_noise=False)

                c_loss = clip_loss(img_gen, text_inputs)
                i_loss = id_loss(img_gen, img_orig)[0] # original
                l2_loss = ((w_start - w_n) ** 2).sum()

                img_for_emotion = F.interpolate(img_gen, size=(112, 112), mode='bilinear', align_corners=False)
                prediction, feat, heads = emotion_model(img_for_emotion)
                e_loss = emotion_loss(prediction, goal_distribution)

                loss = c_loss + lambda_L2_edit * l2_loss + lambda_ID_edit * i_loss + lambda_E_edit * e_loss
                losses.append(loss.item())
                pbar.set_description(f"Loss: {loss.item():.4f}")

                loss.backward()
                optimizer.step()
                scheduler.step()

        final_image = img_gen.detach()

        # Compute ID metric between edited images and original image

        sim = id_loss(final_image, reference_image)[0]
        emo_score = e_loss # TODO: compute emotion score

        sims.append(sim.item())
        emo_scores.append(emo_score.item())

*** Processing image 1/65 ***


Loss: 0.7134: 100%|██████████| 50/50 [00:16<00:00,  3.04it/s]


*** Processing image 2/65 ***


Loss: 0.7236: 100%|██████████| 50/50 [00:16<00:00,  3.02it/s]


*** Processing image 3/65 ***


...

Loss: 0.7485: 100%|██████████| 50/50 [00:16<00:00,  3.02it/s]


*** Processing image 64/65 ***


Loss: 0.7202: 100%|██████████| 50/50 [00:16<00:00,  3.05it/s]


*** Processing image 65/65 ***


Loss: 0.7124: 100%|██████████| 50/50 [00:16<00:00,  3.03it/s]


In [28]:
# Save results

save_path = "results"
os.makedirs(save_path, exist_ok=True)

SAVE_RESULTS = True

if SAVE_RESULTS:
    np.save(os.path.join(save_path, "sims_e4e.npy"), sims_e4e)
    np.save(os.path.join(save_path, "sims_hybrid.npy"), sims_hybrid)
    np.save(os.path.join(save_path, "sims_e4e_edit.npy"), sims_e4e_edit)
    np.save(os.path.join(save_path, "sims_hybrid_edit.npy"), sims_hybrid_edit)
    np.save(os.path.join(save_path, "emo_scores_e4e.npy"), emo_scores_e4e)

In [ ]:
# Load results

LOAD_RESULTS = True

if LOAD_RESULTS:

    sims_e4e = np.load(os.path.join(save_path, "sims_e4e.npy"))
    sims_hybrid = np.load(os.path.join(save_path, "sims_hybrid.npy"))
    sims_e4e_edit = np.load(os.path.join(save_path, "sims_e4e_edit.npy"))
    sims_hybrid_edit = np.load(os.path.join(save_path, "sims_hybrid_edit.npy"))
    emo_scores_e4e = np.load(os.path.join(save_path, "emo_scores_e4e.npy"))

In [32]:
import plotly.express as px
import pandas as pd
import numpy as np

data = {
    "inversion type": ["e4e"] * len(sims_e4e) * 2 + ["hybrid"] * len(sims_hybrid) * 2,
    "operation": ["inversion"] * len(sims_e4e) + ["editing"] * len(sims_e4e) + ["inversion"] * len(sims_hybrid) + ["editing"] * len(sims_hybrid),
    "dissimilarity": (
        sims_e4e +  # Distribution A
        sims_e4e_edit +  # Distribution A
        sims_hybrid + # Distribution B
        sims_hybrid_edit  # Distribution B
    )
}

df = pd.DataFrame(data)

fig = px.box(df, x="inversion type", y="dissimilarity", color="operation")
fig.update_traces(quartilemethod="exclusive") # or "inclusive", or "linear" by default
fig.update_layout(title="Impact of inversion type on editing performance")

fig.show()



In [33]:
import plotly.express as px
import pandas as pd
import numpy as np

data = {
    "inversion type": ["e4e"] * len(sims_e4e) + ["hybrid"] * len(sims_hybrid),
    "emotion loss": (
        emo_scores_e4e + 
        emo_scores_hybrid
    )
}

df = pd.DataFrame(data)

fig = px.box(df, x="inversion type", y="emotion loss", color="inversion type")
fig.update_traces(quartilemethod="exclusive") # or "inclusive", or "linear" by default
fig.update_layout(title="Impact of inversion type on editing performance")

fig.show()